# Stanford SQL Course

This are personal notes about the SQL course offered by Stanford Lagunita.

The url of the course is https://lagunita.stanford.edu/courses/DB/SQL/SelfPaced/course/

In [2]:
# Showing content of repo
!ls

colleges.db  readme.md	stanford_sql_notes.Rmd	stanford_sql_notes.ipynb


In [3]:
# Importing neccessary libraries
import sqlite3
import pandas as pd

In [6]:
# Writting helper functions

db = 'colleges.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    return run_query('''
    select *
    from sqlite_master
    where type in ('table', 'view')
    ''')


In [8]:
# Showing tables
show_tables()

,type,name,tbl_name,rootpage,sql
0,table,College,College,2,"CREATE TABLE College(cName text, state text, e..."
1,table,Student,Student,3,"CREATE TABLE Student(sID int, sName text, GPA ..."
2,table,Apply,Apply,4,"CREATE TABLE Apply(sID int, cName text, major ..."


In [5]:
# Displaying necessary tables
pd.read_sql_query('''
select * from sqlite_master where type in ('table', 'view')
''', conn)

,type,name,tbl_name,rootpage,sql
0,table,College,College,2,"CREATE TABLE College(cName text, state text, e..."
1,table,Student,Student,3,"CREATE TABLE Student(sID int, sName text, GPA ..."
2,table,Apply,Apply,4,"CREATE TABLE Apply(sID int, cName text, major ..."


## Basic Select Statements

In [ ]:
import 